In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
import os
import random
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle

In [2]:
def RandomForest(data):
    data_folder = 'RandomForest/'
    name = data["Dizi İsimleri"]
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    file_name = f"{year}.pkl"
    file_path = os.path.join(data_folder,file_name)
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"RandomForest Prediction for {name}: {prediction}")
    
    
    
def AdaBoost(data):
    data_folder = 'AdaBoost/'
    name = data["Dizi İsimleri"]
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    file_name = f"{year}.pkl"
    file_path = os.path.join(data_folder,file_name)
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"Ada Boost Prediction for {name}: {prediction}")
    
    
def DecisionTree(data):
    data_folder = 'DecisionTree/'
    name = data["Dizi İsimleri"]
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    file_name = f"{year}.pkl"
    file_path = os.path.join(data_folder,file_name)
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"Decision Prediction for {name}: {prediction}")
    

def LightGMB(data):
    data_folder = 'LightGBM/'
    name = data["Dizi İsimleri"]
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    file_name = f"{year}.pkl"
    file_path = os.path.join(data_folder,file_name)
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"LightGBM Prediction for {name}: {prediction}")
    
def NaiveBayes(data):
    data_folder = 'NaiveBayes/'
    name = data["Dizi İsimleri"]
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    file_name = f"{year}.pkl"
    file_path = os.path.join(data_folder,file_name)
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"NaiveBayes Prediction for {name}: {prediction}")

In [4]:
def predict(data):
    RandomForest(data)
    AdaBoost(data)
    DecisionTree(data)
    LightGMB(data)
    NaiveBayes(data)


with open('actor_season_average.txt', 'r') as file:
    dosya_icerigi = file.readlines()

oyuncu_degerleri = {}
for satir in dosya_icerigi:
    oyuncu, deger = satir.split(':')
    oyuncu_degerleri[oyuncu.strip()] = float(deger)

gibi_oyuncular = ["Feyyaz Yiğit","Kıvanç Kılınç","Ahmet Kürşat Öçalan"]
ezel_oyuncular = ["Kenan İmirzalıoğlu","Cansu Dere","Yiğit Özşener","Barış Falay","Tuncel Kurtiz","Haluk Bilginer"]

toplam_gibi = 0
toplam_ezel = 0

for oyuncu in gibi_oyuncular:
    if oyuncu in oyuncu_degerleri:
        toplam_gibi += oyuncu_degerleri[oyuncu]

for oyuncu in ezel_oyuncular:
    if oyuncu in oyuncu_degerleri:
        toplam_ezel += oyuncu_degerleri[oyuncu]

gibi_ortalama = toplam_gibi / len(gibi_oyuncular)
ezel_ortalama = toplam_ezel / len(ezel_oyuncular)


gibi_dizi = {'Dizi İsimleri': 'Gibi','Yapım Yılı':2021,
            'Platform': 'Exxen','Aile':0,'Dram':0,'Komedi':1,'Aksiyon':0,'Polisiye':0,'Tarihi':0,'Romantik':0,
            'Yapım Şirketi': 'OGM Pictures','Gösterim Süresi':40,
            'Başrollerin Sezon Ortalaması':gibi_ortalama,
            'Yönetmen Sezon Ortalaması':1.77,'Senarist Sezon Ortalaması':1.6,'Ödüllü Yönetmen':1,
            'Ödüllü Aktör':1,'İhraç':0,'Ödüllü Dizi':1
            }

ezel_dizi = {'Dizi İsimleri': 'Ezel','Yapım Yılı':2009,
            'Platform': 'Show TV','Aile':0,'Dram':1,'Komedi':0,'Aksiyon':1,'Polisiye':0,'Tarihi':0,'Romantik':0,
            'Yapım Şirketi': 'Ay Yapım','Gösterim Süresi':90,
            'Başrollerin Sezon Ortalaması':ezel_ortalama,
            'Yönetmen Sezon Ortalaması':1.66,'Senarist Sezon Ortalaması':1.33,'Ödüllü Yönetmen':1,
            'Ödüllü Aktör':1,'İhraç':1,'Ödüllü Dizi':1
            }

predict(gibi_dizi)
print("-----")
predict(ezel_dizi)

RandomForest Prediction for Gibi: ['Başarılı']
Ada Boost Prediction for Gibi: ['Başarılı']
Decision Prediction for Gibi: ['Kısmen Olgun']
LightGBM Prediction for Gibi: ['Başarılı']
NaiveBayes Prediction for Gibi: ['Başarılı']
-----
RandomForest Prediction for Ezel: ['Kısmen Başarılı']
Ada Boost Prediction for Ezel: ['Kısmen Başarılı']
Decision Prediction for Ezel: ['Kısmen Başarılı']
LightGBM Prediction for Ezel: ['Kısmen Başarılı']
NaiveBayes Prediction for Ezel: ['Başarılı']
